# Chatbot - Utilizando uma abordagem com RAG

## Introdução

Como referência, serão utilizadas as abordagens apontadas na seção de Retrieval Augmented Generation (RAG) no curso Dell AI Delivery Academy, promovido pelo Instituto Metrópole Digital.

O documento pode ser consultado na página do Dr. Elias Jacob de Menezes Neto, em https://github.com/eliasjacob/dell_deep_learning_genai/blob/main/Notebook_11.ipynb.

## Preparando os dados

### Carregando os dados

Para fins de teste, somente, os dados utilizados serão extraídos do Regimento interno da ALERN, originalmente disponibilizado [neste link](https://www.al.rn.leg.br/regimento-interno/Regimento_Interno_ALRN_junho_2024_DL.pdf). O texto foi extraído para processamento direto, sendo divido em artigos e suas sub-seções.

In [1]:
URL_DADOS = '/content/drive/MyDrive/ALRN-Docs/Chatbot/datasets/regimento_alrn.txt'
URL_CHROMADB = '/content/drive/MyDrive/ALRN-Docs/Chatbot/chroma_db_instructor_xl'

CARREGAR_CHROMADB_EXISTENTE = True
DEVICE = 'cpu'
# DEVICE = 'cuda'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Carregamos, a príncípio, os dados a partir de um arquivo de texto, em que cada linha possui o texto de um dos artigos do Regimento interno, incluindo todas as suas subseções, como parágrafos, alíneas e afins.

In [3]:
import warnings
warnings.filterwarnings("ignore")

with open(URL_DADOS, 'r', encoding='latin-1') as arq:
  texto = arq.read()

# Listagens até 9, em português, são marcadas por números ordinais. Isso faz com que
# posteriormente os números ordinais recebam mais atenção do que deveriam, na
# representação TF-IDF. O código abaixo remove a marcação de ordinais e coloca a
# mesma notação utilizada nos demais itens

for num in range(1, 10):
  texto = texto.replace(f'Art. {num}º', f'Art. {num}.')
  texto = texto.replace(f'art. {num}º', f'art. {num}.')
  texto = texto.replace(f'§ {num}º', f'§ {num}.')

texto = texto.split('\n')
texto.remove('')

### Tratando e formatando os dados

Os dados no conjunto de artigos serão armazenados no ChromaDb, que os armazenará e utilizará um modelo para converter as entradas em vetores semânticos para posterior comparação com as perguntas que servirão de consulta para recuperação dos artigos com potencial de atuar como resposta.

De forma a manter a granularidade dos artigos a serem usados como resposta para as perguntas, o documento será segmentado em fragmentos com no máximo 500 palavras. Assim, os dados podem ser utilizados junto com outras arquiteturas que têm limite no npumero de embeddings, tal qual Transformers. Optamos, portanto, por garantir que cada artigo tenha um máximo de 500 palavras. Nos casos dos artigos que possuem mais do que 500 palavras, dividimo-lo em partes menores, sendo que cada um dos fragmentos do artigo contém em seu início o caput, de forma a manter o contexto do fragmento.

In [4]:
artigos = []
for art in texto:
  item = art.split(' ')
  qtd_palavras = len(item)
  if qtd_palavras > 500:
    item = art.replace('. §', '.\n§').replace('; §', ';\n§').replace(': §', ':\n§').replace(';', '\n').replace(':', '\n').replace('\n ', '\n').replace(' \n', '\n').split('\n')
    caput = item[0]
    fragmento_artigo = '' + caput
    for i in range(1, len(item)):
      if len(fragmento_artigo.split(' ')) + len(item[i]) <= 500:
        fragmento_artigo = fragmento_artigo + ' ' + item[i]
      else:
        artigos.append(fragmento_artigo)
        fragmento_artigo = '' + caput + ' ' + item[i]
    artigos.append(fragmento_artigo)
  else:
    artigos.append(art)

Após a segmentação dos dados, passamos ao seu processamento e armazenamento no ChromaDB.

Trataremos cada um dos artigos/fragmentos de artigo como um documento completo, utilizando as ferramentas da biblioteca `langchain_core.documents`, a saber, `Document`.

In [5]:
!pip install langchain-core

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.6/396.6 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.3/289.3 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [6]:
from langchain_core.documents import Document

# Cria um objeto Document com conteúdo e metadados especificados
# 'page_content' é o conteúdo principal do documento, que é armazenado na variável 'arbitrary_text'
# 'metadata' é um dicionário contendo informações adicionais sobre o documento
# 'title' especifica o título do documento
# 'author' especifica o autor do documento
# 'source' especifica a URL de origem do documento

documentos = []

titulos = []
for artigo in artigos:
  tit = artigo.split('. ')[1]
  titulos.append(tit)
  doc = Document(
    page_content=artigo,
    metadata={
      'title': f'Regimento Interno - Artigo {tit}_{titulos.count(tit)}',
      'author': 'ALERN',
      'source': 'https://www.google.com/url?q=https%3A%2F%2Fwww.al.rn.leg.br%2Fregimento-interno%2FRegimento_Interno_ALRN_junho_2024_DL.pdf'
    }
  )

  documentos.append(doc)

### Inserindo os dados no ChromaDB

Precisaremos dos pacotes `Chroma` incluso em `langchain_chroma` e do `HuggingFaceEmbeddings`, então façamos a instalação, bem como a sua inicialização.

In [7]:
!pip install langchain_chroma chromadb langchain-huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.0/94.0 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.7/149.7 kB 10.5 MB/s eta 0:00:

Visto que o ChromaDB necessita que lhe seja passada uma função de geração de embeddings, utilizaremos o `HuggingFaceEmbeddings` da `lang_chain`, passando o nome do modelo a ser utilizado.

Em várias iterações, utilizamos os modelos abaixo:
- `sentence-transformers` (conforme artigo do Dr Elias)
- `sentence-transformers/multi-qa-mpnet-base-dot-v1` (variação do sentence transformers)
- `hkunlp/instructor-xl` (modelo sugerido pelo artigo do Dr Gabriel Lins - disponível em https://medium.com/@gabrielblins/revolucionando-a-informa%C3%A7%C3%A3o-em-sa%C3%BAde-a-cria%C3%A7%C3%A3o-de-um-chatbot-especialista-utilizando-modelos-de-d246d65d5621)

O `hkunlp/instructor-xl` rendeu melhores resultados, pelo que o mantivemos.

In [8]:
# A classe é utilizada para gerar embeddings usando os modelos no HuggingFace
from langchain_huggingface import HuggingFaceEmbeddings

# Initialize the HuggingFaceEmbeddings with a specified model
# 'model_name' especifica o caminho do modelo pré-treinado Sentence TRansformers
# 'show_progress=True' habilita a exibição do progresso durante o processo de geração de embeddings
# 'model_kwargs' é um dicionário de argumentos para o modelo (nesse caso, especificando o uso de CPU)
embedding_function_instructor_xl = HuggingFaceEmbeddings(
    model_name='hkunlp/instructor-xl',
    show_progress=True,
    model_kwargs={'device': DEVICE}
)

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/66.3k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

In [9]:
from langchain_chroma import Chroma

In [10]:
# Cria uma instância ChromaDB e a salva em um diretório especificado
# 'db_instructor_xl' armazenará a instância ChromaDB criada a partir dos documentos e embeddings

if CARREGAR_CHROMADB_EXISTENTE:
  # Carrega a instância ChromaDB existente a partir do diretório especificado
  # 'persist_directory' especifica o diretório onde a instância ChromaDB será salva
  db_instructor_xl = Chroma(
      persist_directory=URL_CHROMADB,
      embedding_function=embedding_function_instructor_xl
  )
else:
  # Inicializa a instância ChromaDB usando o método 'from_documents'
  # 'documents' é uma lista de fragmentos de documentos que serão armazenados no banco de dados
  # 'embedding' é a função de embedding usada para gerar embeddings dos documentos
  # 'persist_directory' especifica o diretório onde a instância ChromaDB será salva
  db_instructor_xl = Chroma.from_documents(
    documents=documentos,
    embedding=embedding_function_instructor_xl,
    persist_directory=URL_CHROMADB
)

### Testando a instância gerada - consultas

Estando a instância do ChromaDB criada e salva, ela pode ser utilizada para fazer consulta e gerenciar os embeddings dos documentos.

Exemplos de como fazer isso podem ser vistos em https://api.python.langchain.com/en/latest/vectorstores/langchain_chroma.vectorstores.Chroma.html

Para fazer as consultas, dentre várias opções, temos
- `instancia.similarity_search(query="exemplo de fragmento de texto a ser consultado", k=1)`: resulta em uma lista de documentos
- `instancia.similarity_search_with_score(query="qux", k=1)`: resulta em uma lista de documentos com um score de similaridade.

In [11]:
# Define the query string to search for relevant documents
query = 'O que é um deputado?'

# Perform a similarity search using the ChromaDB instance with Sentence Transformers embeddings
# 'docs1' will store the documents that are most similar to the query based on Sentence Transformers embeddings
docs1 = db_instructor_xl.similarity_search_with_score(query)
print(docs1)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[(Document(metadata={'author': 'ALERN', 'source': 'https://www.google.com/url?q=https%3A%2F%2Fwww.al.rn.leg.br%2Fregimento-interno%2FRegimento_Interno_ALRN_junho_2024_DL.pdf', 'title': 'Artigo 102_1'}, page_content='Art. 102. O Deputado só será considerado presente à reunião de Comissão se, em qualquer das fases dos trabalhos, estiver no recinto da mesma. '), 0.3736966550350189), (Document(metadata={'author': 'ALERN', 'source': 'https://www.google.com/url?q=https%3A%2F%2Fwww.al.rn.leg.br%2Fregimento-interno%2FRegimento_Interno_ALRN_junho_2024_DL.pdf', 'title': 'Artigo 344_1'}, page_content='Art. 344. A remuneração do Deputado seguirá o que já dispõe a Constituição do Estado e é devida a partir do início da legislatura ao diplomado antes da instalação da primeira sessão legislativa ordinária; ou a partir da expedição do diploma, ao diplomado posteriormente à instalação; ou a partir da posse, ao Suplente em exercício. § 1. Além do subsídio, o Deputado tem direito a: I \x96 ajuda de custo

Adicionalmente é possível utilizar um `retriever` criado a partir do vector store, utilizando o método `.as_retriever()`. O objeto criado, um retriever, tem a capacidade de consultar e recuperar documetnos relevantes do vector store, funcionando como uma instância executável do vector store, realizando busca com base nos embeddings.

In [12]:
retriever_instructor_xl = db_instructor_xl.as_retriever(search_type='mmr')
# retriever_instructor_xl = db_instructor_xl.as_retriever(search_type='similarity')
# retriever_instructor_xl = db_instructor_xl.as_retriever(search_type='similarity_score_threshold')


In [13]:
query = 'O que é legislatura?'
print(retriever_instructor_xl.invoke(query))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[Document(metadata={'author': 'ALERN', 'source': 'https://www.google.com/url?q=https%3A%2F%2Fwww.al.rn.leg.br%2Fregimento-interno%2FRegimento_Interno_ALRN_junho_2024_DL.pdf', 'title': 'Artigo 3_1'}, page_content='Art. 3. Legislatura é o período correspondente ao mandato parlamentar, de 4 (quatro) anos, iniciando-se em 1º de fevereiro do primeiro ano de mandato e terminando em 31 de janeiro do quarto ano de mandato, dividida em quatro sessões legislativas, uma por ano. '), Document(metadata={'author': 'ALERN', 'source': 'https://www.google.com/url?q=https%3A%2F%2Fwww.al.rn.leg.br%2Fregimento-interno%2FRegimento_Interno_ALRN_junho_2024_DL.pdf', 'title': 'Artigo 134_1'}, page_content='Art. 134. Os Gabinetes Parlamentares são órgãos da Assembleia Legislativa, dotados de autonomia, na forma do § 3. do art. 33-A da Constituição Estadual. '), Document(metadata={'author': 'ALERN', 'source': 'https://www.google.com/url?q=https%3A%2F%2Fwww.al.rn.leg.br%2Fregimento-interno%2FRegimento_Interno_ALR

## Integrando com um LLM

In [14]:
# import transformers
# from transformers import pipeline
# model_name = 'pierreguillou/bert-base-cased-squad-v1.1-portuguese'
# nlp = pipeline("question-answering", model=model_name)pergunta = 'o que é um deputado?'
# respostas = retriever_instructor_xl.invoke(pergunta)
# print (f'respostas {respostas}')
# contextos = [resposta.page_content for resposta in respostas]
# print (f'contextos {" ".join(contextos)}')
# resultado = nlp(question=pergunta, context=' '.join(contextos))
# print (f'resultado {resultado}')

In [15]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [16]:
!nohup ollama serve &> /dev/null &

In [17]:
!ollama pull llama3.1

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest 
pulling 8eeb52dfb3bb...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...   0% ▕▏ 2.9 MB/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...   0% ▕▏  23 MB/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...   1% ▕▏  36 MB/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...   1% ▕▏  63 MB/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...   2% ▕▏  70 MB/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...   2% ▕▏  90 MB/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...   2% ▕▏ 104 MB/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...   3% ▕▏ 119 MB/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...   3% ▕▏ 127 MB/4.7 G

In [18]:
!pip install -U langchain-ollama
from langchain_ollama import ChatOllama

In [19]:
model_llama = ChatOllama(
    model='llama3.1', # 'model' specifies the model to use, in this case 'llama3.1'
    temperature=0, # 'temperature' controls the randomness of the model's output, with 0 being deterministic
    base_url='http://localhost:11434' # 'base_url' specifies the base URL for the model's API endpoint
)

# Invoke the model with a specific prompt/question
response = model_llama.invoke("Oi. Tudo bem?")

# Print the content of the response from the model
print(response.content)

Tudo bem, obrigado! Como posso ajudar você hoje?


In [20]:
response = model_llama.invoke("Oi. Tudo bem?")

# Print the content of the response from the model
print(response.content)

Tudo bem, obrigado! Como posso ajudar você hoje?


In [21]:
# Import necessary classes from the langchain_core module
# ChatPromptTemplate is used to define the structure of the chat prompt
# HumanMessage, SystemMessage, and AIMessage are used to define different types of messages in the chat
# StrOutputParser is used to parse the output of the chat
# RunnablePassthrough is used to pass data through without modification
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Define the chat prompt template with a series of messages
# 'from_messages' method creates a ChatPromptTemplate from a list of message tuples
# Each tuple contains a message type and the message content
prompt = ChatPromptTemplate.from_messages(
    [
        # System message to establish the assistant's role
        # This message sets the context for the assistant, instructing it to answer questions about UFRN
        # If the assistant doesn't know the answer, it should say so
        ("system", "Você é um assistente de alunos que responde a dúvidas sobre a UFRN. Use as informações fornecidas para responder às perguntas dos alunos. Se você não souber a resposta, apenas diga que não sabe."),

        # Placeholder for chat history to maintain context
        # This placeholder will be replaced with the actual chat history during execution
        ("placeholder", "{chat_history}"),

        # Human message placeholder for user input
        # This placeholder will be replaced with the user's question and context during execution
        ("human", "\nCONTEXTO: {context} \n\nPERGUNTA: {question}"),
    ]
)

# Define a function to format retrieved documents
# 'docs' is a list of document objects
# The function joins the page content of each document with four newline characters in between
def format_retrieved_documents(docs):
    return "\n\n\n\n".join([doc.page_content for doc in docs])

In [22]:
# Define a base runnable for the Sentence Transformers retriever
# This runnable will handle the context and question for the chat prompt
base_runnable_instructor_xl = (
    {
        # 'context' key will use the retriever to get relevant documents and format them
        # 'retriever_sentence_transformers' retrieves relevant documents based on the query
        # 'format_retrieved_documents' formats the retrieved documents for the chat prompt
        "context": retriever_instructor_xl | format_retrieved_documents,

        # 'question' key will pass the question directly to the retriever without modification
        "question": RunnablePassthrough(),
    }
    # Combine the context and question with the chat prompt template
    | prompt
)

# Initialize an output parser to parse the string output of the chat
# 'StrOutputParser' is used to parse the output of the chat into a string format
output_parser = StrOutputParser()

In [23]:
rag_chain_sentence_transformers_llama = (
    base_runnable_instructor_xl  # Use the base runnable with Sentence Transformers embeddings
    | model_llama                        # Pass the result to the Llama 3.1 model for response generation
    | output_parser                      # Parse the model's output into a string format
)

In [24]:
rag_chain_sentence_transformers_llama.invoke('O que é uma legislatura?')

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

'Uma legislatura é o período correspondente ao mandato parlamentar da Assembleia Legislativa, que dura 4 anos e inicia-se em 1º de fevereiro do primeiro ano de mandato e termina em 31 de janeiro do quarto ano de mandato.'